First, we do webscraping to scrape poems off of a poet's webpage.

In [ ]:
# imports and downloads
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
from urllib import request
from urllib.request import Request
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pickle
import re

# function to start scraping at a poet's profile
def urlgen():
  # Try different URLs for different responses! But be warned, web scraping takes a few minutes
  #https://mypoeticside.com/poets/oscar-wilde-poems
  #https://mypoeticside.com/poets/william-shakespeare-poems
  #https://mypoeticside.com/poets/maya-angelou-poems
  #https://mypoeticside.com/poets/pablo-neruda-poems
  url = 'https://mypoeticside.com/poets/emily-dickinson-poems'
  return url

# function to scrape text off each page and store in file
def webscrape(link):
    with open("chatbot_data.txt", "a+", encoding='utf-8') as f:
        req = Request(
            url=link,
            headers={'User-Agent': 'Mozilla/5.0'}
        )
        try:
          html = request.urlopen(req).read().decode('utf8')
          soup = BeautifulSoup(html, features='html.parser')
          poem_list = soup.find(class_="list-poems")
          if poem_list is not None:
            links = poem_list.findAll('a')
            results = ["https:"+link.get('href') for link in links]
            for page in results:
              try:
                r = request.urlopen(page).read().decode('utf8')
                soup = BeautifulSoup(r, features='html.parser')
                poem = soup.find(class_='poem-entry')
                if poem is not None:
                  text = poem.find('p').getText()
                  text_chunks = [chunk for chunk in text.splitlines() if not re.match(r'^\s*$', chunk)]
                  for chunk in text_chunks:
                    f.write(chunk)
                    f.write('\n')
              except:
                print("An error occurred")
        except:
          print("An error occurred")

# function to tokenize and find top terms
def preprocess(file_name):
  # tokenize
  text = open(file_name).read()
  text = text.lower()
  tokens = word_tokenize(text)

  # remove stopwords
  stop_words = set(stopwords.words('english'))
  new_tokens = [t for t in tokens if not t in stop_words and len(t) > 3]

  # lemmatize
  wnl = WordNetLemmatizer()
  lemmas = set(wnl.lemmatize(t) for t in new_tokens)
  list_lemmas = list(lemmas)

  # part of speech tagging
  pos_tags = nltk.pos_tag(list_lemmas)
  
  # isolate nouns
  nouns = [n for (n, p) in pos_tags if p.startswith("N")]

  # find most common nouns
  noun_dict = {nouns[i]: new_tokens.count(nouns[i]) for i in range(len(nouns))}
  sort_nouns = sorted(noun_dict.items(), reverse=True, key=lambda x: x[1])
  top = sort_nouns[0:50]
  top_list = [x[0] for x in top]
  t_file = open('terms.txt', 'a+')
  for t in top_list:
    t_file.write(t)
    t_file.write('\n')


# start
url_origin = urlgen()
print(url_origin)

webscrape(url_origin)

f_name = 'chatbot_data.txt'
preprocess(f_name)


In the above code, the preprocess function determines the top 50 terms present in the text. These are the terms we will use to construct the Knowledge Base.

In [3]:
# Knowledge Base construction
# get the top terms
f = open('terms.txt', 'r')
text = f.read()
terms = text.split()
with open('terms.pkl', 'wb') as f:
  pickle.dump(terms, f)
#print(terms)

# get the scraped poem stanzas
f = open('chatbot_data.txt', 'r')
stanzas = f.readlines()
with open('stanzas.pkl', 'wb') as f:
  pickle.dump(stanzas, f)
#print(stanzas[:20])

# construct it
kb = {}

for term in terms:
  for stanza in stanzas:
    if term in stanza:
      if term not in kb:
        kb[term] = [stanza]
      else:
        kb[term].append(stanza)

# for key,value in kb.items():
# 	print(key, ':', value)
 
# pickle it
with open('knowledge_base.pkl', 'wb') as f:
  pickle.dump(kb, f)

Now that we have the knowledge base, we can begin the code that will run the chatbot.

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
import os.path
import string
import random
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


def is_positive(sent) -> bool:
  sia = SentimentIntensityAnalyzer()
  return sia.polarity_scores(sent)["compound"] > 0


def generate_reply(input, terms, stanzas, kb):
  vectorizer = TfidfVectorizer()

  # if there is an exact match with one of the knowledge base keys pick from the knowledge base
  for t in terms:
    if t in input:
      sentences = kb[t]
      sentences.append(input)
      # convert text into numerical vector
      vectors = vectorizer.fit_transform(sentences)
      # compute cosine similarity
      cs = cosine_similarity(vectors[-1], vectors)
      best_index = cs[0].argsort()[-2]
      another = cs[0].argsort()[-4]
      sentences.remove(input)
      if sentences[best_index] == sentences[another]:
        return sentences[best_index]
      else:
        return sentences[best_index] + sentences[another]
  
  # otherwise pick from all stanzas
  stanzas.append(input)
  vectors = vectorizer.fit_transform(stanzas)
  # compute cosine similarity
  cs = cosine_similarity(vectors[-1], vectors)
  best_index = cs[0].argsort()[-2]
  another = cs[0].argsort()[-4]
  stanzas.remove(input)
  if stanzas[best_index] == stanzas[another]:
    return stanzas[best_index]
  else:
    return stanzas[best_index] + stanzas[another]

def run_chatbot():
  # load in pickles
  terms = pickle.load(open('terms.pkl', 'rb'))
  stanzas = pickle.load(open('stanzas.pkl', 'rb'))
  kb = pickle.load(open('knowledge_base.pkl', 'rb'))

  info_phrases =["I think", "I feel", "I like", "I dislike", "I love", "I hate", "I am", "I'm", "I want", "I don't want", "I have", "I don't have", "I know", "I don't know"]
  # initial message
  name = input("ChatPoet: Good day! Welcome to ChatPoet, poetry is a literary art form focused on the aesthetics of language.\n Feeling ruminative? Have a conversation with me! Before we begin, what is your name?\n")

  # load up user model or create one if not
  if os.path.exists('user_model.pkl'):
    user_model = pickle.load(open('user_model.pkl', 'rb'))
  else:
    user_model = {}
  
  # check for returning user
  if name in user_model:
    print("ChatPoet: " + name + "! A pleasure to see you once more. Start talking or press X to exit.")
  else: 
    user_model[name] = []
    print("ChatPoet: It is wonderful to meet you " + name + ". Start talking or press X to exit.")

  # start conversation
  chat = True

  while chat:
    user_said = input("You: ")

    if user_said == "X":
      chat = False
      print("ChatPoet: I bid you adieu.")
    else:
      # hardcoded responses for personal info
      for i in info_phrases:
        if i in user_said:
          # save in user model
          user_model[name].append(user_said)

          # add personalized response for certain phrases
          if i == "I am" or i == "I'm":
            # clean input to reframe
            modify_input = user_said.replace(i, "")
            modify_input = modify_input.rstrip()
            if modify_input[-1] in string.punctuation:
              modify_input = modify_input[:-1]
            
            # perform sentiment analysis
            if is_positive(user_said) > 0:
              print("ChatPoet: You are" + modify_input + "? Lovely to hear!")
            else: 
              print("ChatPoet: You are" + modify_input + "? I'm sorry to hear.")

      # generate response
      response = generate_reply(user_said, terms, stanzas, kb)
      if response != None:
        print("ChatPoet: " + response)
      else: 
        print("ChatPoet: Sorry, I didn't quite understand. Let us converse on a different topic.")
        if name in user_model:
          mention = random.choice(user_model[name])
          print("ChatPoet: Earlier you had told me /'" + mention + "/', how do you feel now?")

  with open('user_model.pkl', 'wb') as f:
    pickle.dump(user_model, f)

  # for key,value in user_model.items():
  #   print(key, ':', value)

# start
run_chatbot()